# TODO
- upgrade pytorch and use `torch.amp.GradScaler` instead of `torch.cuda.amp`
- setup logging dir
- fix dataset code (datasets produce problems for tiny and open)
- should be able to use DataManager.get_dataloaders (as class method)
- improve logging messages

In [24]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from scaling_llms.data import DataManager, DATA_SOURCES
from scaling_llms.models import GPTConfig, GPTModel
from scaling_llms.tracking.managers import RunManager
from scaling_llms.trainer import Trainer, TrainerConfig
from scaling_llms.utils.loggers import setup_root_logging


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
setup_root_logging()

ROOT_DIR = Path("./runs/")
EXPERIMENT_NAME = "gpt_test"

run_path = ROOT_DIR / EXPERIMENT_NAME
run = RunManager.create_new_run_dir(run_path)

In [26]:
BATCH_SIZE = 8

train_cfg = TrainerConfig(
    num_steps=4,
    lr=3e-4,
    accum_steps=2,
    # warmup_steps=3,
    lr_schedule="linear",
    enable_tb=True, 
    train_log_freq=1,
    net_log_freq=2,
    sys_log_freq=3,
    eval_log_freq=-1,
    timer_mode="wall"

)

model_cfg = GPTConfig()
train_dl, val_dl = DataManager().get_loaders(
    source=DATA_SOURCES.tiny_shakespeare,
    seq_len=model_cfg.seq_len,
    batch_size=BATCH_SIZE,
    val_frac=0.1,
    num_workers=0,
)

model = GPTModel(model_cfg)


In [27]:
trainer = Trainer(
    cfg=train_cfg,
    model=model,
    train_dl=train_dl,
    eval_dl=val_dl,
    run=run
)

trainer.train()


starting training
run_dir=runs/gpt_test/run_1
device=cpu precision=fp32
num_steps=4 accum_steps=2 lr=3.000e-04
step=0 | nll=5.5536 | ppl=258.16 | lr=3.000e-04 | tokens_seen=8192
step=1 | nll=4.8612 | ppl=129.17 | lr=2.250e-04 | tokens_seen=16384


KeyboardInterrupt: 